<a href="https://colab.research.google.com/github/xuunnis123/100-Days-Of-ML-Code/blob/master/Cloze_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import os
import glob

In [2]:
! pip install fitz
! pip install nltk
! pip install summa
! pip install rake_nltk
! pip install PyMuPDF
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 10.4 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 500 kB 51.2 MB/s 
     |████████████████████████████████| 421 kB 47.6 MB/s 
     |████████████████████████████████| 130 kB 51.7 MB/s 
     |████████████████████████████████| 5.1 MB 33.6 MB/s 
     |████████████████████████████████| 41 kB 582 kB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=56cd95e7176ba20f4c55af790cd46c8ced4fd01eb387b04bd09a044f78784cb8
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
  Created wheel for pyxnat: filename=pyxnat-1.4-py3-none-any.whl size=92687 sha256=07d6c98e60102faac4cc47e85b7c9d2c55d606d0a058f8864fb8564b6b12cffb
  Stored in directory: /root/.cache/pip/wheels/09/d3/8f/51847ef95ec6448ff7599bf269145f82

In [3]:
import fitz
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import *
from summarizer import Summarizer
from summa import summarizer
from rake_nltk import Rake

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
def is_number(uchar):
    if uchar >= u'\u0030' and uchar<=u'\u0039':
        return True
    else:
        return False

def is_alphabet(uchar):
    if (uchar >= u'\u0041' and uchar<=u'\u005a') or (uchar >= u'\u0061' and uchar<=u'\u007a'):
        return True
    else:
        return False
def is_sign(uchar):
    sign_set = {" ", "-", ".", "(", ")" ,",",":","'","’"}
    if uchar in sign_set:
        return True
    else:
        return False
def is_char(uchar):
    if not (is_alphabet(uchar) or is_number(uchar) or is_sign(uchar)):
        return True
    else:
        return False
    
def read_pdf_text(filename):
    page_content=[]
    text = ""
    with fitz.open("/content/sample_data/"+filename) as pdf:
        page_no = -1
        for page in pdf:
            blocks = page.get_text('blocks')
            blocks = sorted(blocks, key = lambda b: (b[0], b[1]))

            for block_word in blocks:
                if block_word[4].startswith( '<image:'):
                    continue
                elif len( block_word[4].replace('\n','') ) < 10:
                    continue
                page_content.append(block_word[4])

            for page_num in range(len(page_content)):
                for j in page_content[page_num]:
                    if j == '\n':
                        page_content[page_num] = page_content[page_num].replace(j, ' ')
                    if is_char(j):
                        page_content[page_num] = page_content[page_num].replace(j, ' ')
                page_content[page_num] = " ".join(page_content[page_num].split())


            if(page_no < 0):
                for page_num in range(len(page_content)):
                    text = text + page_content[page_num] + '. '
            else:
                text = ''.join(page_content[page_no-1])
            
    text = text[text.find('Introduction')+12:text.find('References')]
    return text

In [5]:
def bert(text):
    model = Summarizer()
    result = model(text, min_length=60)
    summary = ''.join(result)
    return summary

def get_keyword(summary, text):
    # sentence = summarizer.summarize(text_2, ratio=1, split=True)
    sentence = sent_tokenize(summary)
    sentence = list(dict.fromkeys(sentence))

    r = Rake()
    summary = summarizer.summarize(text, ratio=0.5)
    r.extract_keywords_from_text(summary)
    keyword = r.get_ranked_phrases()
    keyword = list(dict.fromkeys(keyword))
    keyword = list(filter(lambda x: len(x) > 6 and len(x.split()) < 3, keyword))[:30]
    return keyword
    # random.shuffle(keyword)

In [ ]:
!ls
!cd sample_data
!ls

sample_data
sample_data


In [6]:
filename = '0_A review of learning analytics intervention in higher education (2011–2018).pdf'
for file in glob.glob('/content/sample_data' + filename):
    head, filename = os.path.split(file)
ks_list = pd.DataFrame(columns = ['sentence', 'key_phrase'])

text = read_pdf_text(filename)
print(text)

. Learning analytics refers to the measurement, collection, analysis, and reporting of data about learners and their contexts, for the purposes of understanding and optimiz- ing learning and the environments in which it occurs (Siemens 2012). Given the massive amount of data available about learners and learning, it has been regarded as having great potential for offering learners a better learning experience that focuses on their individual preferences, strengths and needs (Clow 2013 Siemens 2012). One major objective of learning analytics is to discover early those students who are likely to fail in their studies and provide them with just-in-time and personalised support (Sclater et al. 2016 Wong et al. 2018). Khalil and Ebner (2015) proposed a life cycle of learning analytics which consisted of four stages: (1) data generation usually from learning platforms, such as massive open online courses, learning man- agement systems (LMS) and virtual learning environments (2) tracking trac

In [7]:
summary = bert(text)
print(summary)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (277) found smaller than n_clusters (586). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)


Learning analytics refers to the measurement, collection, analysis, and reporting of data about learners and their contexts, for the purposes of understanding and optimiz- ing learning and the environments in which it occurs (Siemens 2012). Given the massive amount of data available about learners and learning, it has been regarded as having great potential for offering learners a better learning experience that focuses on their individual preferences, strengths and needs (Clow 2013 Siemens 2012). One major objective of learning analytics is to discover early those students who are likely to fail in their studies and provide them with just-in-time and personalised support (Sclater et al. Khalil and Ebner (2015) proposed a life cycle of learning analytics which consisted of four stages: (1) data generation usually from learning platforms, such as massive open online courses, learning man- agement systems (LMS) and virtual learning environments (2) tracking traceable data of learners on 

In [11]:
nltk.download('punkt')
nltk.download('stopwords')
sentences = sent_tokenize(text)
sentences = list(filter(lambda x: len(x.split()) > 7, sentences))
sentences = [re.sub('\n', '', i) for i in sentences]
text = " ".join(sentences)
keyword = get_keyword(summary, text)
print(keyword)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['intervention involved', 'institution ’', 'need ..', 'intervention method', 'commonly exist', 'learning behaviours', 'students ’', 'students via', 'intervention methods', 'particular intervention', 'students may', 'study progress', 'student learning', 'higher education', 'student online', 'student problems', 'learning analytics', 'helping students', 'study information', 'learning data', 'outcome', 'dent demographic', 'online learning', 'learning problems', 'collaborative learning', 'interventions covered', 'intervention strategies', 'task behaviour', 'faculty ’', 'learning progress']


In [12]:
ks_list = pd.DataFrame(columns = ['sentence', 'key_phrase'])
for s in sentences:
    for w in keyword:
        if w in s:
            dic = {'sentence':s,'key_phrase':w}
            ks_list = ks_list.append(dic, ignore_index = True)
            keyword.remove(w)
            sentences.remove(s)
            break
ks_list

,sentence,key_phrase
0,One major objective of learning analytics is t...,learning analytics
1,Although the idea of making learning analytics...,intervention strategies
2,This paper aims to review the case studies of ...,higher education
3,The cases were categorised and summarised acco...,outcome
4,The frequent types of data involved students’ ...,learning behaviours
5,Within the limited availability of empirical s...,intervention method
6,Educ.. are informed by data on the learning pr...,learning progress
7,Wise (2014) asserts that at-risk students may ...,students may
8,The frequent types of data involved students’ ...,online learning
9,Within the limited availability of empirical s...,intervention methods


In [14]:
ks_list.to_csv(filename.split('_')[0] +"_cloze.csv",encoding='utf-8-sig',index=False)